# CreAI TRDD - Real-Time Geographic Information Systems (RTGIS)
#### Agenda:
##### 1. Qué es RTGIS?
##### 2. Qué soluciona?
##### 3. Qué propone?
##### 4. Dónde se aplica?
##### 5. Herramientas populares para RTGIS
##### 5. Caso de uso.

## Metadatos y preparación de dispositivo y Puntos de Interés (POIs)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import plotly.express as px

import os
from dotenv import load_dotenv

load_dotenv()

USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")

from pyicloud import PyiCloudService
api = PyiCloudService(USER, PASSWORD)

cel = api.devices[0]

device_status = cel.status()
cel_red_status = int(device_status["deviceStatus"])
dev_name = "Oswaldo Paez"
lat_osw = 19.38108785952171
long_osw = -99.11358114095226

# if cel_red_status > 200:
#     print("Error: ", cel_red_status)
#     exit()

battery_pencetage = round(device_status["batteryLevel"], 2)
cel_location = cel.location()
cel_latitude = cel_location["latitude"]
cel_longitude = cel_location["longitude"]
cel_timestamp = cel_location["timeStamp"]

current_location = Point(cel_longitude, cel_latitude)




In [ ]:
# Places where I usually go when I go whatever place. Add whatever you want.
eje5_corner = Point(-99.114088,19.379680)
eje3_corner = Point(-99.110596, 19.381173)
apatlaco_corner = Point(-99.113562, 19.381633)

gdf_poi = gpd.GeoDataFrame({'geometry': [apatlaco_corner, eje3_corner, eje5_corner], "POI Nombre": ["Esquina Apatlaco", "Esquina Eje 3", "Esquina Eje 5"]}, crs="EPSG:4326")
gdf_poi["type"] = "POI"
gdf_poi_meters = gdf_poi.to_crs(epsg=3857)
gdf_poi_meters["buffer"] = gdf_poi_meters.buffer(100)

## Implementación y uso

In [1]:
mode = "DEMO" # "DEBUG" or "DEMO"

In [2]:
# import plotly.graph_objects as go
import plotly.graph_objs as go
import numpy as np
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from datetime import datetime
import dash
import random
import dash_bootstrap_components as dbc

# initialise dataframe and figure
# gdf = gpd.GeoDataFrame({f"{dev_name} - {battery_pencetage}%": current_location}, crs="EPSG:4326")
gdf = gpd.GeoDataFrame(
    [{
        f"geometry": current_location,
        "dev_name": dev_name,
        "battery": battery_pencetage,
        "timestamp": cel_timestamp,
        "type": "User",
        "poi_index": None
    }],
    crs="EPSG:4326")

gdf_meters = gdf.to_crs(epsg=3857)

print(gdf_meters)


fig = go.Figure(data=px.scatter_map(
    data_frame=gdf, 
    lat=gdf.geometry.y, 
    lon=gdf.geometry.x,
    hover_name=f"dev_name",
    hover_data=["battery", "timestamp"],
    color="battery",
    zoom=14, 
    # height=600,
    # mapbox_style="open-street-map",
    # marker={"size": 10, "opacity": 0.7, "color": "blue", 'symbol': ["harbor"]},
    title="<b>CreAI</b> - Demo RTGIS",
    range_color=[0.0, 1.0],
    size=[0.1],
    center={"lat": lat_osw, "lon": long_osw},
)
)

fig.add_trace(
    px.scatter_map(
        data_frame=gdf_poi, 
        lat=gdf_poi.geometry.y, 
        lon=gdf_poi.geometry.x,
        hover_name="POI Nombre",
        color="type",
        size=[0.01, 0.01, 0.01]
    ).data[0]
)




fig.update_layout(
    dragmode='drawclosedpath',
    newshape={
        "line_color": "blue",
        "fillcolor": "navy",
        "opacity": 0.5,
    },
    autosize=True
)


# Build App
# app = JupyterDash(__name__)
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.layout = html.Div(
    [
        # dcc.Graph(id="creai-map", figure=fig, style={"height": 920, "width": "100%"}),
        dcc.Graph(id="creai-map", figure=fig, style={"responsive": True, "height": 760}),
        dcc.Interval(id="animateInterval", interval=5000, n_intervals=0),
        html.Br(),
        html.Div(id='msg-creai', children=[
            html.Div("Messages:", style={"font-weight": "bold", "font-size": "16px"}),
        ])
    ],
)


# Function to get the index of the POI within the buffer
def get_poi_index(point, buffers):
    for idx, buffer in buffers.items():
        if buffer.contains(point):
            return idx
    return None

@app.callback(
    Output("creai-map", "figure"),
    Output("msg-creai", component_property='children'),
    Input("animateInterval", "n_intervals"),
    State("creai-map", "figure"),
    State("msg-creai", "children"),
)
def doUpdate(i, fig, div_old_msgs_children):
    global gdf
    current_location = None
    msg_creai = ''

    if mode == "DEMO":
        device_status = cel.status()
        cel_red_status = int(device_status["deviceStatus"])

        # if cel_red_status > 200:
        #     print("Error: ", cel_red_status)
        #     exit()

        battery_percetage = round(device_status["batteryLevel"], 2)
        cel_location = cel.location()
        cel_latitude = cel_location["latitude"]
        cel_longitude = cel_location["longitude"]
        cel_timestamp = cel_location["timeStamp"]
        gdf["battery"] = battery_percetage
        gdf["timestamp"] = cel_timestamp

        current_location = Point(cel_longitude, cel_latitude)
    else:
        # Simulate movement
        range = 0.002
        random_x = random.uniform(-range, range)
        random_y = random.uniform(-range, range)
        current_location = Point(long_osw + random_x, lat_osw + random_y)

    gdf["geometry"] = current_location
    gdf_meters = gdf.to_crs(epsg=3857)


    is_within = gdf_meters.geometry.apply(
        lambda x: gdf_poi_meters["buffer"].contains(x).any()
    )

    # Apply the function to get the POI index for each dynamic point
    gdf_meters['poi_index'] = gdf_meters.geometry.apply(
        lambda x: get_poi_index(x, gdf_poi_meters['buffer'])
    )


    for n, row in gdf_meters.iterrows():
        if row['poi_index'] is not None:
            cel.display_message('', f"Hola {row['dev_name']}! Tu batería está al {battery_pencetage * 100}% y estás cerca de {gdf_poi_meters.iloc[row['poi_index']]['POI Nombre']}.")
            message = f"Usuario {row['dev_name']} está cerca de {gdf_poi_meters.iloc[row['poi_index']]['POI Nombre']}"
            msg_creai = f"[{datetime.now()}] Enviando msj a dispositivo: {message}"
            print(message)
            print(gdf_poi_meters.distance(row.geometry))
            print(is_within)

    updated_fig = go.Figure( px.scatter_map(
        data_frame=gdf, 
        lat=gdf.geometry.y, 
        lon=gdf.geometry.x,
        hover_name="dev_name",
        hover_data=["battery", "timestamp"],
        color="battery",
        zoom=15, 
        # height=600,
        # mapbox_style="open-street-map",
        # marker={"size": 10, "opacity": 0.7, "color": "blue", 'symbol': ["harbor"]},
        title=f"<b>CreAI</b> - Demo RTGIS - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        range_color=[0.0, 1.0],
        size=[0.001],
        center={"lat": lat_osw, "lon": long_osw}
    )
    )

    updated_fig.add_trace(
        px.scatter_map(
            data_frame=gdf_poi, 
            lat=gdf_poi.geometry.y, 
            lon=gdf_poi.geometry.x,
            hover_name="POI Nombre",
            color="type",
            size=[0.01, 0.01, 0.01]
        ).data[0]
    )

    updated_fig.update_layout(
        dragmode='drawclosedpath',
        newshape={
            "line_color": "blue",
            "fillcolor": "navy",
            "opacity": 0.5,
        },
        autosize=True
    )

    gdf_meters['poi_index'] = None
    return updated_fig, div_old_msgs_children +[html.Div(msg_creai)]


if __name__ == '__main__':
    app.run_server(mode="external", debug=True, port=8050)


KeyboardInterrupt

